Since I didn't get acces to the twitter API in time for the deadline, I worked only on sample 1 (that was given to me by Trung). For all the questions regarding sample 2 in the for I put 0 as a default value, sorry for the incovenence.

In [ ]:
import pymongo
import json

In [56]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [57]:
db = client['csm']
collection = db['tweets']

In [ ]:
#read data file
#list_tweets=[]
#posts = db.posts
#with open('covidtrack.json', 'r', encoding='utf8') as f:
#    json.load(f)
#    for line in f:
#        print('1')
#        data = json.load(line)
#        list_tweets.append(data)
#        
#    for tw in list_tweets:
#        print('2')
#       posts.insert_one(tw)


# --->>>Since for some reason this didn't work I imported the data directly trhough the MongoDB Compass Community software <---

## 1

In [117]:
#1
#Percentage of Tweets that contain URLs

url_percentage = 1-collection.count_documents({"entities.urls":[]})/collection.estimated_document_count()
print('Percentage of Tweets that contain URLs: '+str(url_percentage*100)+' percents.')

Percentage of Tweets that contain URLs: 17.766666666666666 percents.


## 2

In [118]:
#2
#Percentage of Tweets that are (or contain) retweets.

retweet_percentage = collection.count_documents({"$or":[{"quoted_status":{"$exists": True }},{"retweeted_status":{"$exists": True}}]})/collection.estimated_document_count()
print('Percentage of Tweets that contain URLs: '+str(retweet_percentage*100)+' percents.')

Percentage of Tweets that contain URLs: 82.39999999999999 percents.


## 3

In [116]:
#3
#Distribution of languages declared in the tweets (%EN, %FR, etc.)
from collections import OrderedDict

languages = collection.distinct('lang')
total_tweets = collection.estimated_document_count()
languages_percentage = {}
for lan in languages:
    languages_percentage[lan]= collection.count_documents({"lang":lan})/total_tweets
    #print(str(languages_percentage[lan]*100)+' percents of tweets are in'+str(lan))
    
sorted_lan = sorted(((value, key) for (key,value) in languages_percentage.items()), reverse=True)
for item in sorted_lan:
    print(str(item[0]*100)+' percents of tweets are in '+str(item[1]))

63.46666666666667 percents of tweets are in en
12.9 percents of tweets are in es
9.700000000000001 percents of tweets are in fr
3.6333333333333337 percents of tweets are in it
3.5333333333333337 percents of tweets are in und
1.0666666666666667 percents of tweets are in ca
0.9666666666666667 percents of tweets are in hi
0.8 percents of tweets are in pt
0.7333333333333333 percents of tweets are in nl
0.7000000000000001 percents of tweets are in tr
0.36666666666666664 percents of tweets are in mr
0.36666666666666664 percents of tweets are in de
0.26666666666666666 percents of tweets are in ta
0.26666666666666666 percents of tweets are in ar
0.23333333333333336 percents of tweets are in te
0.23333333333333336 percents of tweets are in in
0.13333333333333333 percents of tweets are in ru
0.13333333333333333 percents of tweets are in ja
0.1 percents of tweets are in th
0.06666666666666667 percents of tweets are in ur
0.06666666666666667 percents of tweets are in tl
0.06666666666666667 percent

## 4

In [148]:
#Table of the 20 most frequent hashtags in the following format [rank, hashtag, frequency]
hashtags = collection.distinct('entities.hashtags.text')
aggr=collection.aggregate([
    {"$unwind": "$entities.hashtags"},
    {"$group" : {"_id" :"$entities.hashtags.text","count":{"$sum":1}}},
    {"$sort": {"count":-1}}
])
table_hashtags=[]
i=1;
for hashtag in aggr:
    table_hashtags.append([i,hashtag['_id'],hashtag['count']])
    i+=1;
#print(table_hashtags[0:19])
for i in range(0,20):
    print(table_hashtags[i])


[1, 'coronavirus', 3002]
[2, 'COVID19', 281]
[3, 'covid19', 61]
[4, 'China', 28]
[5, 'Covid_19', 28]
[6, 'COVIDー19', 26]
[7, 'AFP', 26]
[8, 'ULTIMAHORA', 25]
[9, 'HKtensions', 25]
[10, 'COVID', 24]
[11, 'CoronavirusOutbreak', 23]
[12, 'lockdown', 23]
[13, 'Inde', 18]
[14, 'cuarentena', 17]
[15, 'coronavirusmemes', 17]
[16, 'coronapocalypse', 16]
[17, 'Cuba', 16]
[18, 'StayAtHome', 15]
[19, 'Pontivy', 12]
[20, 'BREAKING', 11]


## 5

In [155]:
#chosen mainly from https://socialblade.com/twitter/top/100/followers and https://www.ngoadvisor.net/
id_ngo = ["24019986","1159936093","1133819209","15536990","14254687","17044207","918202831","63504620","36688694","19394213","198858818","140708237","209575740","71579643","467253439","46396292","280386299","1408406798","13201312","18213483"]
id_media = ["972651","428333","18735898","14075928","14293310","6017542","5402612","2557521","14800270","2890961","816653","1344951","3108351","15736190","2707054218","14780915","836692559531552769","759251","11883132","3064617094"]


#Percentage of Tweets directly generated by all the 20 media accounts together.
media_percentage = collection.count_documents({"user.id_str":{"$in":id_media}})/collection.estimated_document_count()

#Percentage of Tweets directly generated by the 20 NGO/gov. accounts
ngo_percentage = collection.count_documents({"user.id_str":{"$in":id_ngo}})/collection.estimated_document_count()

#Percentage of Tweets generated by all of the 20 media accounts that appear as retweets in the sample.
media_retweet_percentage = collection.count_documents({"$or":[{"retweeted_status.user.id_str":{"$in":id_media}},{"quoted_status.id_str":{"$in":id_media}}]})/collection.estimated_document_count()

#Percentage of Tweets generated by all of the 20 NGO/gov. accounts that appear as retweets in the sample.
ngo_retweet_percentage = collection.count_documents({"$or":[{"retweeted_status.user.id_str":{"$in":id_ngo}},{"quoted_status.id_str":{"$in":id_ngo}}]})/collection.estimated_document_count()


print('Percentage of Tweets directly generated by the 20 NGO accounts: '+str(ngo_percentage*100))
print('Percentage of Tweets generated by all of the 20 media accounts that appear as retweets in the sample: '+str(media_percentage*100))
print('Percentage of Tweets generated by all of the 20 media accounts that appear as retweets in the sample: '+str(media_retweet_percentage*100))
print('Percentage of Tweets generated by all of the 20 NGO/gov. accounts that appear as retweets in the sample: '+str(ngo_retweet_percentage*100))

Percentage of Tweets directly generated by the 20 NGO accounts: 0.0
Percentage of Tweets generated by all of the 20 media accounts that appear as retweets in the sample: 0.0
Percentage of Tweets generated by all of the 20 media accounts that appear as retweets in the sample: 0.0
Percentage of Tweets generated by all of the 20 NGO/gov. accounts that appear as retweets in the sample: 0.0


## 6
#Discuss in a couple of lines the results of points 4 & 5: 
#Are the top hashtags expected or surprising? Why? Are any of the percentages unexpected? Why?



As expected, 9 out of the 20 most frequente hashtags where directely mentioning the coronavirus.
What is suprising is that all the tweets out of 3000 had the hashtag "coronavirus", but only around 400 had the hashtag with the official name (covid-19).
A few of them are mentioning the media or the country they comme from and an other bunch are about the lockdown.
The only surpising hashtag is the one about the Honk Kong tension. It would mean that the outbreak of the covid-19 and the politcal situation in Honk Kong could be related.

For section 5, it is really surprising that none of the ngo or major madia where found in the sample, specialy because around 80% percents of the tweets are retweets.
My guess is that the sample is not big enough. The socund hypothesis is the time when the tweets where collected. Since the major media tweet accounts are from america, if it was collected when it was night there there wouldn't be any tweets from them.